In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/gemstone.csv')

In [4]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df = df.drop(labels=['id'],axis=1)

In [7]:
x = df.drop(labels=['price'],axis=1)
y = df[['price']]

In [8]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [29]:
#define which columns should be ordinal-encoded and which should be scaled
categorical_cols = x.select_dtypes(include='object').columns
numerical_cols = x.select_dtypes(exclude='object').columns

In [30]:
#define the custom ranking for each ordinal variable
cut_categories =['Fair','Good','very Good','premium','Ideal']
color_categories =['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [31]:
from sklearn.impute import SimpleImputer #Handling Missing values
from sklearn.preprocessing  import StandardScaler # Handling feature scaling
from sklearn.preprocessing import OrdinalEncoder # ordinal Encoding
# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 

In [32]:
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Categorical pipeline with unknown handling
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories],
                                   handle_unknown='use_encoded_value', unknown_value=-1)),
        ('scaler', StandardScaler())  # Optional if numerical scaling is desired post-encoding
    ]
)

# Combine both pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_cols),
        ('cat', cat_pipeline, categorical_cols)
    ]
)


In [38]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Define your category lists (ensure these are predefined correctly)
cut_categories = ['Fair','Good','very Good','premium','Ideal']  # Define your categories for 'cut'
color_categories = ['D','E','F','G','H','I','J']  # Define your categories for 'color'
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF'] # Define your categories for 'clarity'

# Numerical pipeline
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Categorical pipeline with unknown handling
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories],
                                   handle_unknown='use_encoded_value', unknown_value=-1)),
        ('scaler', StandardScaler())  # Optional if numerical scaling is desired post-encoding
    ]
)

# Combine both pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_cols),
        ('cat', cat_pipeline, categorical_cols)
    ]
)

# Fit and transform the training data
x_train_transformed = preprocessor.fit_transform(x_train)

# Debug: Check the shape of transformed data
print(f"Transformed shape: {x_train_transformed.shape}")

# Adjust the feature names based on the shape
num_features = numerical_cols
# This assumes each categorical column is encoded as one feature
# Adjust if your encoder produces multiple features per column
cat_features = [f'{col}_encoded' for col in categorical_cols] 

# Ensure the lengths match
if len(num_features) + len(cat_features) != x_train_transformed.shape[1]:
    cat_features = [f'{col}_encoded_{i}' for i, col in enumerate(categorical_cols) for _ in range(1)]  # Adjust this range if needed

feature_names = num_features + cat_features

# Convert the transformed data into a DataFrame
x_train = pd.DataFrame(x_train_transformed, columns=feature_names)

# Transform the test data
x_test_transformed = preprocessor.transform(x_test)
x_test = pd.DataFrame(x_test_transformed, columns=feature_names)


Transformed shape: (135501, 9)


ValueError: operands could not be broadcast together with shapes (6,) (3,) 

In [39]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=30)

In [34]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns=preprocessor.get_feature_names_out())

AttributeError: Estimator imputer does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?